In [8]:
import requests
from pprint import pprint
import pandas as pd
import pymysql
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import uuid
sns.set()
%matplotlib inline

In [9]:
db = pymysql.connect(host='localhost', user='root', database='admin_main', port = 3307, charset='utf8mb4', cursorclass=pymysql.cursors.DictCursor)

# Create a cursor object
cursorObject = db.cursor()

messages_Query = "select * from messages"
messages_meta_Query =  "select * from messages_meta"

cursorObject.execute(messages_Query)
cursorObject.execute(messages_meta_Query)

messages_df = pd.read_sql(messages_Query, db)
messages_meta_df = pd.read_sql(messages_meta_Query, db)

In [10]:
messages_new = messages_df.drop(messages_df.columns.difference(['id', 'conversation_id', 'session_id','recipient_id', 'sender_id', 'type', 'incoming', 'created_at']), 1)
messagesmeta_new = messages_meta_df.drop(messages_meta_df.columns.difference(['message_id', 'value']), 1)

In [11]:
messages_count = messages_df.drop(messages_df.columns.difference(['incoming', 'created_at']), 1)

messages_count.insert(loc=2, column='year_month_day', value=messages_count['created_at'].map(lambda x: 10000*x.year + 100*x.month + x.day))
messages_count.insert(loc=3, column='month', value=messages_count.created_at.dt.month)
# +1 to make Monday=1.....until Sunday=7
messages_count.insert(loc=4, column='day', value=messages_count.created_at.dt.dayofweek+1)
messages_count.insert(loc=5, column='hour', value=messages_count.created_at.dt.hour)

messages_count = messages_count.groupby(['year_month_day', 'incoming']).size()
messages_count

year_month_day  incoming
20191113        0           3828
                1           2446
20191114        0           3284
                1           1944
20191115        0           3425
                            ... 
20200209        1           2685
20200210        0           3811
                1           2453
20200211        0           3580
                1           2022
Length: 182, dtype: int64

In [12]:
import ast
messagesmeta_new.rename(columns={'message_id': 'id'}, inplace=True)

messages = pd.merge(messages_new, messagesmeta_new, on='id', how='inner')

messages_new = messages[messages.recipient_id == 0]
messages_new = messages_new[messages_new.type == 'postback']

invalid_otp = messages[messages.sender_id == 0]
invalid_otp = messages[messages['value'].str.contains("You have entered an invalid OTP code.")]

messages_new['value'] = messages_new['value'].apply(ast.literal_eval)

# c = 1
# for k in messages_new.value:
#     messages_new['new_value'] = k[c]['title']
#     c+=1

messages_new['new_value'] = [d.get('payload') for d in messages_new.value]

# Slice by Date and Activities

# start_date = "2019-11-20"
# end_date = "2019-12-01"
# messages_new = messages_new[messages_new.created_at.between(start_date, end_date)]

# list = ['Get Started', 'Continue', 'Add More Products', 'Add to My Bag']

# messages_new = messages_new[messages_new['new_value'].isin(list)]

messages_new 

,id,conversation_id,session_id,recipient_id,sender_id,type,incoming,created_at,value,new_value
5,6,ef22edab-a532-471b-99ff-9b4be241866e,1,0,1,postback,1,2019-11-13 06:20:13,"{'title': 'Select', 'payload': 'FULL_MENU'}",FULL_MENU
8,9,8d4f77fd-a807-4f21-b796-ea285ac7ad80,2,0,2,postback,1,2019-11-13 06:20:16,"{'title': 'Continue', 'payload': 'POLICY_AGREED'}",POLICY_AGREED
10,11,ef22edab-a532-471b-99ff-9b4be241866e,1,0,1,postback,1,2019-11-13 06:20:30,"{'title': 'Select', 'payload': '{""type"": ""cate...","{""type"": ""category_items"", ""category_id"":7, ""p..."
12,13,ef22edab-a532-471b-99ff-9b4be241866e,1,0,1,postback,1,2019-11-13 06:21:02,"{'title': 'Select', 'payload': '{""id"":33,""name...","{""id"":33,""name"":""Burger McDo"",""quantity"":1,""pr..."
14,15,ef22edab-a532-471b-99ff-9b4be241866e,1,0,1,postback,1,2019-11-13 06:21:08,"{'title': 'Add to My Bag', 'payload': '{""id"":3...","{""id"":33,""name"":""Burger McDo"",""quantity"":1,""pr..."
...,...,...,...,...,...,...,...,...,...,...
547960,547969,55339c69-c8b3-42f6-a1d9-1d78f8f9ef55,42233,0,25092,postback,1,2020-02-11 23:01:17,"{'title': 'Select', 'payload': 'FULL_MENU'}",FULL_MENU
547964,547973,55339c69-c8b3-42f6-a1d9-1d78f8f9ef55,42233,0,25092,postback,1,2020-02-11 23:01:28,"{'title': 'Select', 'payload': '{""type"": ""cate...","{""type"": ""category_items"", ""category_id"":8, ""p..."
547966,547975,55339c69-c8b3-42f6-a1d9-1d78f8f9ef55,42233,0,25092,postback,1,2020-02-11 23:01:36,"{'title': 'Select', 'payload': '{""id"":15,""name...","{""id"":15,""name"":""1-pc. Spicy Chicken McDo w/ R..."
547970,547979,55339c69-c8b3-42f6-a1d9-1d78f8f9ef55,42233,0,25092,postback,1,2020-02-11 23:01:54,"{'title': 'Select', 'payload': '{""id"":15,""name...","{""id"":15,""name"":""1-pc. Spicy Chicken McDo w/ R..."


In [13]:
messages_new = messages_new.drop(messages_new.columns.difference(['created_at', 'new_value']), 1)

In [ ]:
start_date = "2020-03-01 00:00:00"
end_date = "2020-03-31 23:59:59"
messages_count = messages_counts[message_counts.created_at.betweem(start_date,end_date)]
messages_new = messages_new[messages_new.created_at.betweem(start_date,end_date)]
invalid_otp = invalid_otp[invalid_otp.created_at.betweem(start_date,end_date)]


In [14]:
writer = pd.ExcelWriter("/Users/AF/Downloads/Mcdo data/Mc Messages.xlsx", engine='xlsxwriter')

messages_count.to_excel(writer, sheet_name='Messages Count')
messages_new.to_excel(writer, sheet_name='Payload')
invalid_otp.to_excel(writer, sheet_name='Invalid OTP')

writer.save()